# **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note** If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

In [ ]:
#reading in an image
image = mpimg.imread('test_images/solidWhiteRight.jpg')
#printing out some stats and plotting
print('This image is:', type(image), 'with dimesions:', image.shape)
plt.imshow(image)  #call as plt.imshow(gray, cmap='gray') to show a grayscaled image

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [2]:
import math

def readImageIn(pathToImage):
    """Reading in an image"""
    image = mpimg.imread(pathToImage)
    return image
    
def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    
    The input to this is an array of line endpoints, derived from a HoughLinesP
    transform. Rather than paint a large number of short stroke lines using the endpoints,
    we are roughly looking at the slope values and then segregating them into left lane and
    right lane lines. Not all endpoints are being considered as a line with a given
    slope can appear either in left or right lane. The 500 number below is the midpoint
    for the x-axis length of the input image, and so we are eliminating endpoints
    with certain slopes that are generally expected to be in the left or right lane.
    The 0.25/0.75 slopes appear reasonable given the general nature of the lane slopes.
    Once the two subsets of left lane and right lane endpoints are available, we extract
    the most extreme endpoints out of them and paint a line only once.
    """
    right_lane_values = []
    left_lane_values = []
    for line in lines:
        for x1,y1,x2,y2 in line:
            if (x2 != x1):
                cv2.line(img, (x1, y1),(x2, y2), color, thickness)
                if x2>=500 and ((y2-y1)/(x2-x1)) > 0.25 and ((y2-y1)/(x2-x1)) < 0.75:
                    right_lane_values.append(populateTempCoordinateArray(x1,y1))
                    right_lane_values.append(populateTempCoordinateArray(x2,y2))
                if x2<500 and ((y2-y1)/(x2-x1)) > -0.75 and ((y2-y1)/(x2-x1)) < -0.25:
                    left_lane_values.append(populateTempCoordinateArray(x1,y1))
                    left_lane_values.append(populateTempCoordinateArray(x2,y2))
                    
    data= np.array(right_lane_values)  
    if len(data) > 0:
        paintExtrapolatedLines (data, img, color, thickness)

    data= np.array(left_lane_values)
    if len(data) > 0:
        paintExtrapolatedLines (data, img, color, thickness)
        
def paintExtrapolatedLines (arrayData, img, color, thickness):
    """
    Given an array of x,y endpoints, extrapolate a line to constrained end points.
    Then paint the lines on a given img.
    """
    fit = np.polyfit(arrayData[:,0],arrayData[:,1],1)  
    lane_extrapolated_begin_values_y = 320
    lane_extrapolated_begin_values_x = \
        int((lane_extrapolated_begin_values_y - fit[1])/fit[0])
    lane_extrapolated_end_values_y = 540
    lane_extrapolated_end_values_x = \
        int((lane_extrapolated_end_values_y - fit[1])/fit[0])
    cv2.line(img, (lane_extrapolated_begin_values_x,
       lane_extrapolated_begin_values_y),
        (lane_extrapolated_end_values_x,
         lane_extrapolated_end_values_y), color, thickness)    
    
def populateTempCoordinateArray (x, y):
    """
    Helper function that returns a single array of x/y coordinate.
    Its used later to create an array of arrays.
    """
    current_coordinates = []
    current_coordinates.append(x)
    current_coordinates.append(y)
    return current_coordinates
    
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Output "lines" is an array containing endpoints of detected line segments
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]),
            minLineLength=min_line_len, maxLineGap=max_line_gap)
    return lines

# Python 3 has support for cool math symbols.
def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

# Convenience function to process file , given path

def processImageUsingFilePath(pathToImageAsString):
    # Read in the file from its path, given as string
    imageIn = readImageIn(pathToImageAsString)
    return processImageUsingFileObject(imageIn)

# Begin defining the Main image processing function
    
def processImageUsingFileObject(imageIn):
    # Grayscale the image
    imageGray = grayscale(imageIn)
    
    # Define a kerne size and apply Gaussian smoothing
    kernel_size = 5;
    imageBlurGray = gaussian_blur(imageGray, kernel_size)

    # Define our parameters for Canny and apply
    low_threshold = 125
    high_threshold = 250
    imageCanny = canny(imageBlurGray, low_threshold, high_threshold)

    # Define a polygon to mask
    left_bottom = [0, 540]
    left_top = [450, 320]
    right_top = [495, 320]
    right_bottom = [1000, 540]
    polygon = np.array([ left_bottom, left_top,right_top, right_bottom], np.int64)
    imageRegionOfInterest = region_of_interest(imageCanny, [polygon])
    
    # Define the Hough transform parameters
    rho = 1 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 2     # minimum number of votes (intersections in Hough grid cell)
    min_line_len = 5 #minimum number of pixels making up a line
    max_line_gap = 5    # maximum gap in pixels between connectable line segments
    # Run Hough on edge detected image
    # Output "lines" is an array containing endpoints of detected line segments
    lines = hough_lines(imageRegionOfInterest, rho, theta, threshold, min_line_len,
            max_line_gap)

    # creating a copy of the imageRegionOfInterest to draw lines on.
    line_image = np.zeros((*imageRegionOfInterest.shape, 3), dtype=np.uint8)
    
    # Iterate over the output "lines" and draw lines on the imageRegionOfInterest
    color = [255,0,0]
    thickness = 6
    draw_lines(line_image, lines, color, thickness)

    # Weight the original image and the line image appropriately
    lines_edges = weighted_img(imageIn, line_image,0.8, 1, 1) 
        
    # Visually inspect the result for effect
    # plt.imshow(lines_edges)
    # processed image files are in /test_images/processed_images
    # processed videos are in the main folder (same folder as this notebook)
    
    return lines_edges
    
# Run/Test the main image processing function
# processImageUsingFilePath('test_images/solidYellowCurve.jpg');
    

## Test on Images

Now you should build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [ ]:
import os
os.listdir("test_images/")

run your solution on all test_images and make copies into the test_images directory).

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

In [3]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
def process_image(imageIn):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image with lines are drawn on lanes)
    
    # This function is unused as i am simply calling processImageUsingFileObject defined previously 
    # in the next two cells.
            
    return result

Let's try the one with the solid white lane on the right first ...

In [4]:
white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(processImageUsingFileObject) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video white.mp4
[MoviePy] Writing video white.mp4


100%|█████████▉| 221/222 [00:02<00:00, 85.68it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: white.mp4 

CPU times: user 5.75 s, sys: 1.09 s, total: 6.83 s
Wall time: 2.94 s


Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [5]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

**At this point, if you were successful you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform.  Modify your draw_lines function accordingly and try re-running your pipeline.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [6]:
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(processImageUsingFileObject)
%time yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video yellow.mp4
[MoviePy] Writing video yellow.mp4


100%|█████████▉| 681/682 [00:08<00:00, 81.48it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: yellow.mp4 

CPU times: user 13.4 s, sys: 2.7 s, total: 16.1 s
Wall time: 8.76 s


In [7]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Reflections

Congratulations on finding the lane lines!  As the final step in this project, we would like you to share your thoughts on your lane finding pipeline... specifically, how could you imagine making your algorithm better / more robust?  Where will your current algorithm be likely to fail?

Please add your thoughts below,  and if you're up for making your pipeline more robust, be sure to scroll down and check out the optional challenge video below!

- No unit tests have been provided.
- Haven't been able to research why vidoes produced by simply calling the earlier written functions are slightly more jumpy, than if code is again copy pasted here.
- Some parameters like the slopes within which we expect each lane to be appear a tad non-scientific.
- Videos have quite a bit of jump , on the left lane extrapolated line, towards the end of videos. More smoothing was expected.
- wasn't able to copy processed images using shutil or similar. Insated manually copied those over.


## Submission

If you're satisfied with your video outputs it's time to submit!


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [ ]:
challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))